# Beyond Classical Search

This notebook serves as supporting material for the chapter **Beyond Classical Search**. The notebooks illustrate the use of the code repository and demonstrate how the code can be extended to solve various search-related problems. Classical Search (Chapter 3) addresses a single category of problems: _observable_, _deterministic_, _known environments_ where the solution is a sequence of actions. Here, we look at what happens when these assumptions are relaxed. The discussion begins with the **local search** on state space. Then we examine what happens if we relax the assumptions of determinism and observability. At last, we'll study **online search**, in which the agent is faced with a state space that is initially unknown and must be explored. 

In [24]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

## Local Search Algorithms and Optimization Problems

These algorithms are suitable for problems in which all that matters is the solution state, not the path cost to reach it. For example, in the 8-queens problem, what matters is the final configuration of queens, not the order in which they were added. Therefore, **Local search** algorithms operate using a single **current node** (rather than multiple paths) and generally move only to the neighbors of that node i.e. the paths followed by the search are not retained. In addition to finding goals, local search algorithms are useful for solving pure **optimization problems**, in which the aim is to find the best state according to an **objective function**.

To understand local search, we consider the **state-space landscape**. A landscape has both _location_ (defined by the state) and _elevation_ (defined by the value of heuristic cost function or objective function). If elevation corresponds to cost, the aim is to find the **global minimum**; if elevation corresponds to an objective function, the aim is to find the **global maximum**.

### Hill-climbing Search

It is simply a loop that continually moves in the direction of increasing value. It terminates when it reaches a "peak" where no neighbor has a higher value. It does not maintain a search tree, so the data structure for the current node need only record the state and value of objective function. Therefore, this algorithm does not look ahead beyond the immediate neighbors of the current state.

Hill climbing is sometimes called **greedy local search** because it grabs a good neighbor state without thinking ahead where to go next. It chooses randomly among the set of best successors if there are more than one. Unfortunately, hill climbing often gets stuck for the following reasons:-
* **Local maxima**: A local maxima is a peak that is higher than each of its neighboring states but lower than the global maximum. Hill-climbing algorithms that reach the vicinity of a local maximum will be drawn upwards towards the peak but will then be stuck with nowhere else to go.
* **Ridges**: Ridges result in a sequence of local maxima that is very difficult for greedy algorithms to navigate.
* **Plateaux**: A plateau is a flat area of the state space landscape. 

Let's have a look at the pseudo code of hill-climbing search.

In [25]:
%%python
from notebookUtils import *
pseudocode('Hill Climbing')

### AIMA4e

__function__ HILL-CLIMBING(_problem_) __returns__ a state that is a local maximum  
&emsp;_current_ &larr; _problem_.INITIAL\-STATE  
&emsp;__loop do__  
&emsp;&emsp;&emsp;_neighbor_ &larr; a highest\-valued successor of _current_  
&emsp;&emsp;&emsp;_if_ VALUE(_neighbour_) &le; VALUE(_current_) __then return__ _current_  
&emsp;&emsp;&emsp;_current_ &larr; _neighbor_  

---
__Figure 4.2__ The hill-climbing search algorithm, which is the most basic local search technique. At each step the current node is replaced by the best neighbor.

## AIMA3e
__function__ HILL-CLIMBING(_problem_) __returns__ a state that is a local maximum  
&emsp;_current_ &larr; MAKE\-NODE(_problem_.INITIAL\-STATE)  
&emsp;__loop do__  
&emsp;&emsp;&emsp;_neighbor_ &larr; a highest\-valued successor of _current_  
&emsp;&emsp;&emsp;__if__ _neighbor_.VALUE &le; _current_.VALUE __then return__ _current_.STATE  
&emsp;&emsp;&emsp;_current_ &larr; _neighbor_

---
__Figure ??__ The hill\-climbing search algorithm, which is the most basic local search technique. At each step the current node is replaced by the best neighbor; in this version, that means the neighbor with the highest VALUE, but if a heuristic cost estimate _h_ is used, we would find the neighbor with the lowest _h_.

The implementation of the above pseudo code can be viewed [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/search/local/HillClimbingSearch.java). 

This algorithm halts if it reaches a plateau where the best successor has the same value as the current state. It might be a good idea to allow sideways move in the hope that plateau is really a shoulder, but we must take care. An infinite loop can occur whenever the algorithm reaches a flat local maximum that is not a shoulder. One common solution is to put a limit on the number of consecutive sideways moves allowed.

### Random-Restart Hill Climbing

The hill climbing algorithms described so far are incomplete as they often get stuck on local maxima. Therefore, **Random-restart hill climbing** conducts a series of hill climbing searches from randomly generated initial states, until a goal is found. If each hill climbing has a probability $p$ of success, then the expected number of restarts required is $1/p$. The success of hill climbing depends very much on the shape of the state space landscape: if there are few local maxima and plateaux, random restart hill climbing will find a good solution very quickly.  

### Simulated Annealing

A hill climbing algorithm that _never_  makes "downhill" moves toward states with lower values is guaranteed to be incomplete, because it can get stuck on a local maximum. In contrast, a purely random walk is complete but extremely inefficient. Therefore we need to combine hill climbing with a random walk in some way that yields both efficiency and completeness. **Simulated annealing** is such an algorithm.

Instead of picking the _best_ move, it picks a _random_ move. If the move improves the situation, it is always accepted. Otherwise, the algorithm accepts the move with some probability less than 1. Suppose $ΔE$ is the amount by which evaluation is worsened ($ΔE<0$) and $T$ is the temperature such that: "bad" moves are more likely to be allowed at high $T$ and they become more unlikely as $T$ decreases. Then the probability decreases exponentially with the "badness" of the move. Mathematically, the probability of accepting the move worsening the evaluation function is given by $e^{\tfrac{\Delta E}{T}}$.  

Let's have a look at the pseudo code of simulated annealing.

In [26]:
%%python
from notebookUtils import *
pseudocode('Simulated Annealing')

### AIMA4e  

__function__ SIMULATED-ANNEALING(_problem_,_schedule_) __returns__ a solution state  

&emsp;_current_ &larr; _problem_.INITIAL\-STATE  
&emsp;__for__ _t_ = 1 __to__ &infin;  __do__  
&emsp;&emsp;&emsp;_T_ &larr; _schedule(t)_  
&emsp;&emsp;&emsp;__if__ _T_ = 0 __then return__ _current_  
&emsp;&emsp;&emsp;_next_ &larr; a randomly selected successor of _current_  
&emsp;&emsp;&emsp;_&Delta;E_ &larr; VALUE(_next_) - VALUE(_current_)  
&emsp;&emsp;&emsp;__if__ _&Delta;E_ > 0 __then__ _current_ &larr; _next_  
&emsp;&emsp;&emsp;__else__ _current_ &larr; _next_ only with probability e<sup>_&Delta;E_/_T_</sup>

---
__Figure 4.6__ The simulated annealing algorithm, a version of stochastic hill climbing where
some downhill moves are allowed. The schedule input determines the value of the “temper-
ature” T as a function of time; higher temperatures early in the schedule mean that downhill
moves are accepted more readily; late in the schedule with low temperatures, downhill moves
are mostly rejected.


## AIMA3e
__function__ SIMULATED-ANNEALING(_problem_,_schedule_) __returns__ a solution state  
&emsp;__inputs__: _problem_, a problem  
&emsp;&emsp;&emsp;&emsp;_schedule_, a mapping from time to "temperature"  

&emsp;_current_ &larr; MAKE\-NODE(_problem_.INITIAL\-STATE)  
&emsp;__for__ _t_ = 1 __to__ &infin;  __do__  
&emsp;&emsp;&emsp;_T_ &larr; _schedule(t)_  
&emsp;&emsp;&emsp;__if__ _T_ = 0 __then return__ _current_  
&emsp;&emsp;&emsp;_next_ &larr; a randomly selected successor of _current_  
&emsp;&emsp;&emsp;_&Delta;E_ &larr; _next_.VALUE - _current_.VALUE  
&emsp;&emsp;&emsp;__if__ _&Delta;E_ > 0 __then__ _current_ &larr; _next_  
&emsp;&emsp;&emsp;__else__ _current_ &larr; _next_ only with probability e<sup>_&Delta;E_/_T_</sup>

---
__Figure__ ?? The simulated annealing algorithm, a version of stochastic hill climbing where some downhill moves are allowed. Downhill moves are accepted readily early in the annealing schedule and then less often as time goes on. The _schedule_ input determines the value of the temperature _T_ as a function of time.

The implementation of the above pseudo code can be viewed [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/search/local/SimulatedAnnealingSearch.java). 

Let's find out the probability of accepting a move at 2 different temperatures. As per the algorithm, bad moves are more likely to be accepted at a higher temperature. 

In [28]:
import aima.core.agent.Action;
import aima.core.search.local.SimulatedAnnealingSearch;

SimulatedAnnealingSearch<String, Action> search = new SimulatedAnnealingSearch<>(null);
		int deltaE = -1; //bad move as ΔE<0
		double lowerTemperature = 1.0;
		double higherTemperature = 20.0;
		
		System.out.println("Probability of acceptance of move at lower temperature: " + search.probabilityOfAcceptance(lowerTemperature, deltaE));
		System.out.println("Probability of acceptance of move at higher temperature: " + search.probabilityOfAcceptance(higherTemperature, deltaE));

Probability of acceptance of move at lower temperature: 0.36787944117144233
Probability of acceptance of move at higher temperature: 0.951229424500714


null

### Local beam search

This algorithm keeps track of $k$ states rather than just one. It begins with $k$ randomly generated states. At each step, all the successors of all $k$ states are generated. If anyone is a goal then algorithm halts. Otherwise, it selects the $k$ best successors from the complete list and repeats. 

Note that the two algorithms i.e. random-restart search and local beam search are quite different. In a random restart search, each search process runs independently of the others. _In a local beam, useful information is passed among the parallel search threads_. In effect, the algorithm quickly abandons unfruitful searches and moves its resources to where the most progress has been made. 

### Genetic Algorithms

